In [1]:
import pandas as pd 

In [2]:
import numpy as np
import glob
import os

In [3]:
emb_glob_regex = '../results/embeddings/concepts_embeddings_*_biobert.csv'

In [4]:
def precision(TP, FP): 
    return TP/(TP+FP)

def recall(TP, FN): 
    return TP/(TP+FN)

def F1(precision, recall):
    return 2/(1/precision + 1/recall)

In [5]:
def return_TP_FN_FP(keywords, ground_truth):
    keywords = set(keywords)
    ground_truth = set(ground_truth)
    TP = len(keywords.intersection(ground_truth) )
    FN = len(keywords.difference(ground_truth))
    FP = len(ground_truth.difference(keywords))
    return np.array([TP, FN, FP])

In [6]:
def explode_list(pmid, pairs):
    result = []
    for pair in pairs:
        result.append((pmid, pair[0], pair[1]))
    return result

def explode_df(df): 
    result = []
    for i, row in df.iterrows(): 
        result += explode_list(row['PMID'], eval(row['ncbo_annotations_pairs']))
    return pd.DataFrame(result, columns=['PMID', 'keyword', 'tag'])

In [7]:
gt = pd.read_csv('../data/concepts_per_document.csv')\
    .rename({'Concepts': 'true_CUIs'}, axis=1)\
    .drop('Unique_concepts', axis=1)
gt['true_CUIs'] = gt['true_CUIs'].apply(eval).apply(set).apply(list).apply(lambda cuis: [cui[5:] for cui in cuis])
gt.head()

,PMID,true_CUIs
0,25763772,"[C4308010, C0854135, C0599755, C0026882, C0597..."
1,25847295,"[C0150312, C0229671, C0009968, C1383501, C0007..."
2,26316050,"[C0005847, C0444498, C1708386, C0442069, C2828..."
3,26406200,"[C0231472, C0026845, C0277814, C1257890, C0224..."
4,26424709,"[C0022558, C2939193, C0241311, C0018704, C0679..."


In [26]:
concepts_emb = pd.concat([pd.read_csv(file) for file in glob.glob(emb_glob_regex)]).reset_index(drop=True)

In [27]:
concepts_to_cuis = concepts_emb.groupby('concept_name').apply(lambda x: list(x['CUI'])).reset_index()

In [28]:
concepts_to_cuis = concepts_to_cuis.rename({0: 'CUIs'}, axis=1)

In [29]:
del concepts_emb

In [13]:
def concat_cuis(x): 
    result = {
        'pred_cuis': sum(x['CUIs'], []),
        'true_cuis': sum(x['true_CUIs'], [])
    }
    return pd.Series(result, index=['pred_cuis', 'true_cuis'])

In [18]:
UMLS_ST21pv_semantic_types_ids = {'T005', 'T007', 'T017', 'T022', 'T031', 'T033', 'T037', 'T038',
'T058', 'T062', 'T074', 'T082', 'T091', 'T092', 'T097', 'T098', 'T103', 'T168', 'T170', 'T201', 'T204'}

In [19]:
s_types = pd.read_csv('../data/MRSTY.RRF', sep='|', header=None, dtype=str)
s_types = s_types[[0, 1]].rename({0: 'CUI', 1: 'type'}, axis=1)
s_types = s_types.loc[s_types['type'].isin(UMLS_ST21pv_semantic_types_ids)]
s_types = s_types.groupby('CUI').apply(lambda x: list(x['type'])).reset_index()\
    .rename({0: 'types'}, axis=1)
s_types.head()

,CUI,types
0,C0000722,[T170]
1,C0000723,[T170]
2,C0000731,[T033]
3,C0000734,[T033]
4,C0000742,[T005]


In [20]:
s_types = dict(zip(s_types['CUI'], s_types['types']))


In [37]:
def change_cui_to_type(x):
    result = {}
    result['pred_type'] = [s_types[cui] for cui in set(x['pred_cuis']) if cui in s_types]
    result['true_type'] = [s_types[cui] for cui in set(x['true_cuis']) if cui in s_types]
    
    result['pred_type'] = sum(result['pred_type'], [])
    result['true_type'] = sum(result['true_type'], [])

    return pd.Series(result, index=['pred_type', 'true_type'])


In [116]:
import copy
stats = []
for file in glob.glob(r'..\results\emb_tagger\tagged_1*'):
    tagged = explode_df(pd.read_csv(file))
    tagged = tagged.merge(concepts_to_cuis, left_on='tag', right_on='concept_name', how='left')\
        .merge(gt, how='left')
    
    tagged = tagged.groupby('PMID').apply(concat_cuis).reset_index()
    
    tagged[['pred_type', 'true_type']] = tagged[['pred_cuis','true_cuis']].apply(change_cui_to_type, axis=1)
    
    TP_FN_FP_cuis = np.zeros((3,))
    TP_FN_FP_types = np.zeros((3,))
    for i, row in tagged.iterrows(): 
        TP_FN_FP_cuis += return_TP_FN_FP(row['pred_cuis'], row['true_cuis'])
        TP_FN_FP_types += return_TP_FN_FP(row['pred_type'], row['true_type'])
    
    precision_ = precision(TP_FN_FP_cuis[0], TP_FN_FP_cuis[2])
    recall_ = recall(TP_FN_FP_cuis[0], TP_FN_FP_cuis[1])
    stats.append(('cui', os.path.basename(file), precision_, recall_, F1(precision_, recall_)))
    
    precision_ = precision(TP_FN_FP_types[0], TP_FN_FP_types[2])
    recall_ = recall(TP_FN_FP_types[0], TP_FN_FP_types[1])
    stats.append(('type', os.path.basename(file), precision_, recall_, F1(precision_, recall_)))

pd.DataFrame(stats, columns=['agg_level', 'file', 'precision', 'recall', 'F1']).sort_values(['agg_level', 'file'])

,agg_level,file,precision,recall,F1
4,cui,tagged_1_word_LDA_test_2022-12-07_15-04-23_emb...,0.016789,0.026272,0.020486
6,cui,tagged_1_word_LDA_train_2022-12-07_15-04-23_em...,0.014899,0.022509,0.017930
0,cui,tagged_1_word_bertopic_lemmatize_nostopwords_d...,0.012911,0.019117,0.015413
2,cui,tagged_1_word_bertopic_lemmatize_nostopwords_d...,0.011797,0.017724,0.014166
5,type,tagged_1_word_LDA_test_2022-12-07_15-04-23_emb...,0.478951,0.443718,0.460662
7,type,tagged_1_word_LDA_train_2022-12-07_15-04-23_em...,0.423714,0.402200,0.412677
1,type,tagged_1_word_bertopic_lemmatize_nostopwords_d...,0.359718,0.390211,0.374344
3,type,tagged_1_word_bertopic_lemmatize_nostopwords_d...,0.361691,0.377584,0.369467


In [117]:
"""n = 300
random = pd.DataFrame()
random['true'] = [np.random.choice(21,size=(6,),replace=False) for _ in range(n)]
random['pred'] = [np.random.choice(21,size=(8,),replace=False) for _ in range(n)]
TP_FN_FP_random = np.zeros((3,))
for i, row in random.iterrows(): 
    TP_FN_FP_types += return_TP_FN_FP(row['true'], row['true'])

precision_ = precision(TP_FN_FP_types[0], TP_FN_FP_types[2])
recall_ = recall(TP_FN_FP_types[0], TP_FN_FP_types[1])
precision_, recall_, F1(precision_, recall_)"""

"n = 300\nrandom = pd.DataFrame()\nrandom['true'] = [np.random.choice(21,size=(6,),replace=False) for _ in range(n)]\nrandom['pred'] = [np.random.choice(21,size=(8,),replace=False) for _ in range(n)]\nTP_FN_FP_random = np.zeros((3,))\nfor i, row in random.iterrows(): \n    TP_FN_FP_types += return_TP_FN_FP(row['true'], row['true'])\n\nprecision_ = precision(TP_FN_FP_types[0], TP_FN_FP_types[2])\nrecall_ = recall(TP_FN_FP_types[0], TP_FN_FP_types[1])\nprecision_, recall_, F1(precision_, recall_)"

In [118]:
import copy
for file in glob.glob('../results/bertopic_nbco/disambiguation/*disambiguation.csv') + glob.glob('../results/lda_ncbo/disambiguation/*'):
    df = pd.read_csv(file)
    df = df[['PMID', 'disambiguation_fine_tuned_best_concept']]
    df['disambiguation_fine_tuned_best_concept'] = df['disambiguation_fine_tuned_best_concept']\
        .apply(eval).apply(lambda x: list(x.values()))
    df = df.explode('disambiguation_fine_tuned_best_concept')
    df = df.merge(concepts_to_cuis, left_on='disambiguation_fine_tuned_best_concept', right_on='concept_name', how='left')\
        .drop('disambiguation_fine_tuned_best_concept', axis=1)\
        .merge(gt, how='left')\
        .rename({'CUIs': 'pred_cuis', 'true_CUIs': 'true_cuis'}, axis=1)
    df['pred_cuis'] = df['pred_cuis'].apply(lambda x: [] if x != x else x)
    df[['pred_type', 'true_type']] = df[['pred_cuis','true_cuis']].apply(change_cui_to_type, axis=1)
    
    TP_FN_FP_cuis = np.zeros((3,))
    TP_FN_FP_types = np.zeros((3,))
    for i, row in df.iterrows(): 
        TP_FN_FP_cuis += return_TP_FN_FP(row['pred_cuis'], row['true_cuis'])
        TP_FN_FP_types += return_TP_FN_FP(row['pred_type'], row['true_type'])
    
    precision_ = precision(TP_FN_FP_cuis[0], TP_FN_FP_cuis[2])
    recall_ = recall(TP_FN_FP_cuis[0], TP_FN_FP_cuis[1])
    stats.append(('cui', os.path.basename(file), precision_, recall_, F1(precision_, recall_)))
    
    precision_ = precision(TP_FN_FP_types[0], TP_FN_FP_types[2])
    recall_ = recall(TP_FN_FP_types[0], TP_FN_FP_types[1])
    stats.append(('type', os.path.basename(file), precision_, recall_, F1(precision_, recall_)))
stats = pd.DataFrame(stats, columns=['agg_level', 'file', 'precision', 'recall', 'F1'])
stats.sort_values(['agg_level', 'file'])#.to_csv('../results')

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_3552\269410256.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  return 2/(1/precision + 1/recall)


,agg_level,file,precision,recall,F1
8,cui,bertopic_ncbo_fulldata_lemmatize_nostopwords_d...,0.000106,0.029283,0.000212
10,cui,bertopic_ncbo_fulldata_lemmatize_nostopwords_d...,0.000106,0.024451,0.000211
12,cui,ncbo_lda_test_21_results_2022-12-08.csv,0.000000,0.000000,0.000000
14,cui,ncbo_lda_train_21_results_2022-12-08.csv,0.000034,0.000692,0.000064
4,cui,tagged_1_word_LDA_test_2022-12-07_15-04-23_emb...,0.016789,0.026272,0.020486
6,cui,tagged_1_word_LDA_train_2022-12-07_15-04-23_em...,0.014899,0.022509,0.017930
0,cui,tagged_1_word_bertopic_lemmatize_nostopwords_d...,0.012911,0.019117,0.015413
2,cui,tagged_1_word_bertopic_lemmatize_nostopwords_d...,0.011797,0.017724,0.014166
9,type,bertopic_ncbo_fulldata_lemmatize_nostopwords_d...,0.004637,0.574675,0.009200
11,type,bertopic_ncbo_fulldata_lemmatize_nostopwords_d...,0.004613,0.497829,0.009141


In [119]:
files_to_names = {'../results/bertopic_nbco/disambiguation\\bertopic_ncbo_fulldata_lemmatize_nostopwords_data_2022-11-30_12-09-18_test_disambiguation.csv': 'bertopic_ncbo_test',
 '../results/bertopic_nbco/disambiguation\\bertopic_ncbo_fulldata_lemmatize_nostopwords_data_2022-11-30_12-09-18_train_disambiguation.csv': 'bertopic_ncbo_train',
 '../results/lda_ncbo/disambiguation\\ncbo_lda_test_21_results_2022-12-08.csv': 'lda_ncbo_test',
 '../results/lda_ncbo/disambiguation\\ncbo_lda_train_21_results_2022-12-08.csv': 'lda_ncbo_train',
 '..\\results\\emb_tagger\\tagged_1_word_bertopic_lemmatize_nostopwords_data_2022-11-28_19-41-48_test_emb_tagger_biobert.csv':'bertopic_emb_test',
 '..\\results\\emb_tagger\\tagged_1_word_bertopic_lemmatize_nostopwords_data_2022-11-28_19-41-48_train_emb_tagger_biobert.csv': 'bertopic_emb_train',
 '..\\results\\emb_tagger\\tagged_1_word_LDA_test_2022-12-07_15-04-23_emb_tagger_biobert.csv': 'lda_emb_test',
 '..\\results\\emb_tagger\\tagged_1_word_LDA_train_2022-12-07_15-04-23_emb_tagger_biobert.csv': 'lda_emb_train'}
files_to_names = {os.path.basename(file): name for file, name in files_to_names.items()}

In [120]:
stats['file'] = stats['file'].map(files_to_names)

In [121]:
for col in ['precision','recall','F1']: 
    stats[col] = stats[col].apply(lambda x: round(100*x, 2))

In [122]:
stats.sort_values(['agg_level', 'file']).to_csv('../results/stats.csv', index=False)

In [124]:
stats.sort_values(['agg_level', 'file'])

,agg_level,file,precision,recall,F1
0,cui,bertopic_emb_test,1.29,1.91,1.54
2,cui,bertopic_emb_train,1.18,1.77,1.42
8,cui,bertopic_ncbo_test,0.01,2.93,0.02
10,cui,bertopic_ncbo_train,0.01,2.45,0.02
4,cui,lda_emb_test,1.68,2.63,2.05
6,cui,lda_emb_train,1.49,2.25,1.79
12,cui,lda_ncbo_test,0.00,0.00,0.00
14,cui,lda_ncbo_train,0.00,0.07,0.01
1,type,bertopic_emb_test,35.97,39.02,37.43
3,type,bertopic_emb_train,36.17,37.76,36.95
